# Suggestion: Investigate Spatial Econometrics

As seen from trying to make the models, our 3 attempts at a normal regression model does not predict the realSum with a high accuracy. In this notebook,  we will try to validate the reason for that. For regression to work, there are some assumptions in place, one of them being the data is homoskedastic. This will be done using pysal's OLS.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
import spreg
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_csv('rome_weekdays.csv')
raw_data.head()

,Index,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,156.874664,Private room,False,True,2,True,1,0,10,95,1,2.978468,1.595733,281.163932,6.230648,697.727246,15.191486,12.48654,41.92498
1,1,172.772543,Private room,False,True,2,False,1,0,9,80,1,0.935371,0.649269,482.707193,10.696887,1251.524333,27.249208,12.49627,41.90801
2,2,277.745307,Entire home/apt,False,False,4,False,0,1,9,90,1,2.203025,0.494697,691.708998,15.328408,1625.897266,35.400361,12.47700,41.90700
3,3,444.906834,Entire home/apt,False,False,6,False,1,0,9,92,2,2.703010,1.295153,805.592641,17.852092,2035.819533,44.325522,12.46969,41.90019
4,4,131.391298,Private room,False,True,3,False,1,0,9,91,1,1.295968,0.867455,317.076369,7.026475,836.622814,18.215634,12.51544,41.89463


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4492 entries, 0 to 4491
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Index                       4492 non-null   int64  
 1   realSum                     4492 non-null   float64
 2   room_type                   4492 non-null   object 
 3   room_shared                 4492 non-null   bool   
 4   room_private                4492 non-null   bool   
 5   person_capacity             4492 non-null   int64  
 6   host_is_superhost           4492 non-null   bool   
 7   multi                       4492 non-null   int64  
 8   biz                         4492 non-null   int64  
 9   cleanliness_rating          4492 non-null   int64  
 10  guest_satisfaction_overall  4492 non-null   int64  
 11  bedrooms                    4492 non-null   int64  
 12  dist                        4492 non-null   float64
 13  metro_dist                  4492 

# Data Preparation

## Data Cleaning

Removing outliers

In [4]:
## Checking the initial shape of data
model_predictors = ['guest_satisfaction_overall', 'person_capacity', 'dist', 'metro_dist', 'room_type']
model_variables = ['realSum', 'guest_satisfaction_overall', 'person_capacity', 'dist', 'room_type', 'lng', 'lat']
numeric_variables = ['realSum', 'guest_satisfaction_overall', 'person_capacity', 'dist']

model_data = pd.DataFrame(raw_data[model_variables])
print("Initial shape: ", model_data.shape)
print("")

## Counting the outliers
def find_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    IQR = q3-q1
    outliers = (df > (q3 + 1.5 * IQR)).sum() + (df < (q1 - 1.5 * IQR)).sum()
    return outliers

max_outliers = 0
for var in numeric_variables:
    print(var)
    print("outliers: ", find_outliers(model_data[var]))
    max_outliers += find_outliers(model_data[var])
    
print("")
print("Max outliers: ", max_outliers)

Initial shape:  (4492, 7)

realSum
outliers:  226
guest_satisfaction_overall
outliers:  299
person_capacity
outliers:  0
dist
outliers:  82

Max outliers:  607


In [5]:
#cleaning
def outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    IQR = q3 - q1
    
    lower = q1 - 1.5 * IQR
    upper = q3 + 1.5 * IQR
    
    ls = df.index[(df[col] < lower) | (df[col] > upper)] #getting index of outliers
    return ls

def remove(df, ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

index_clearing = []
index_clearing.extend(outliers(model_data, 'realSum'))
        
model_data_cleaned = pd.DataFrame(remove(model_data, index_clearing))
print("New shape: ", model_data_cleaned.shape)

New shape:  (4266, 7)


## Making pandas geoframe

In [6]:
final_model_data = model_data_cleaned
final_model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4266 entries, 0 to 4491
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   realSum                     4266 non-null   float64
 1   guest_satisfaction_overall  4266 non-null   int64  
 2   person_capacity             4266 non-null   int64  
 3   dist                        4266 non-null   float64
 4   room_type                   4266 non-null   object 
 5   lng                         4266 non-null   float64
 6   lat                         4266 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 266.6+ KB


In [7]:
final_model_data["geometry"] = gpd.points_from_xy(final_model_data["lng"], final_model_data["lat"])
final_model_data = gpd.GeoDataFrame(final_model_data, crs="epsg:4326")

# Check the first rows
final_model_data.head()

,realSum,guest_satisfaction_overall,person_capacity,dist,room_type,lng,lat,geometry
0,156.874664,95,2,2.978468,Private room,12.48654,41.92498,POINT (12.48654 41.92498)
1,172.772543,80,2,0.935371,Private room,12.49627,41.90801,POINT (12.49627 41.90801)
2,277.745307,90,4,2.203025,Entire home/apt,12.47700,41.90700,POINT (12.47700 41.90700)
4,131.391298,91,3,1.295968,Private room,12.51544,41.89463,POINT (12.51544 41.89463)
5,182.124237,89,4,1.285514,Entire home/apt,12.51535,41.89470,POINT (12.51535 41.89470)


# Test for Heteroskedasticity Using Pysal OLS

Now that we have cleaned and encoded the data, we shall try making a linear regression model.  


In [8]:
final_num_predictors = ['guest_satisfaction_overall','person_capacity', 'dist']

In [16]:
rome_OLS = spreg.OLS(final_model_data[['realSum']].values, final_model_data[final_num_predictors].values, 
                  name_y = 'realSum', name_x = final_num_predictors)

print(rome_OLS.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:        4266
Mean dependent var  :    183.9032                Number of Variables   :           4
S.D. dependent var  :     66.2901                Degrees of Freedom    :        4262
R-squared           :      0.2882
Adjusted R-squared  :      0.2877
Sum squared residual:13340892.032                F-statistic           :    575.1649
Sigma-square        :    3130.195                Prob(F-statistic)     :  6.979e-314
S.E. of regression  :      55.948                Log likelihood        :  -23219.389
Sigma-square ML     :    3127.260                Akaike info criterion :   46446.779
S.E of regression ML:     55.9219                Schwarz criterion     :   46472.212

-----------------------------------------------------------------------------

# Analysis for London

In [10]:
london_data = pd.read_csv('london_weekdays.csv')

## Cleaning Data

In [11]:
## Checking the initial shape of data
model_predictors = ['guest_satisfaction_overall', 'person_capacity', 'dist', 'metro_dist', 'room_type']
model_variables = ['realSum', 'guest_satisfaction_overall', 'person_capacity', 'dist', 'room_type', 'lng', 'lat']
numeric_variables = ['realSum', 'guest_satisfaction_overall', 'person_capacity', 'dist']

model_data = pd.DataFrame(london_data[model_variables])
print("Initial shape: ", model_data.shape)
print("")

## Counting the outliers
def find_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    IQR = q3-q1
    outliers = (df > (q3 + 1.5 * IQR)).sum() + (df < (q1 - 1.5 * IQR)).sum()
    return outliers

max_outliers = 0
for var in numeric_variables:
    print(var)
    print("outliers: ", find_outliers(model_data[var]))
    max_outliers += find_outliers(model_data[var])
    
print("")
print("Max outliers: ", max_outliers)

Initial shape:  (4614, 7)

realSum
outliers:  247
guest_satisfaction_overall
outliers:  176
person_capacity
outliers:  0
dist
outliers:  129

Max outliers:  552


In [12]:
#cleaning
def outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    IQR = q3 - q1
    
    lower = q1 - 1.5 * IQR
    upper = q3 + 1.5 * IQR
    
    ls = df.index[(df[col] < lower) | (df[col] > upper)] #getting index of outliers
    return ls

def remove(df, ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

index_clearing = []
index_clearing.extend(outliers(model_data, 'realSum'))
        
london_data_cleaned = pd.DataFrame(remove(model_data, index_clearing))
print("New shape: ", london_data_cleaned.shape)

New shape:  (4367, 7)


## Making pandas geoframe

In [13]:
london_model_data = london_data_cleaned
london_model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4367 entries, 0 to 4613
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   realSum                     4367 non-null   float64
 1   guest_satisfaction_overall  4367 non-null   float64
 2   person_capacity             4367 non-null   float64
 3   dist                        4367 non-null   float64
 4   room_type                   4367 non-null   object 
 5   lng                         4367 non-null   float64
 6   lat                         4367 non-null   float64
dtypes: float64(6), object(1)
memory usage: 272.9+ KB


In [14]:
london_model_data["geometry"] = gpd.points_from_xy(london_model_data["lng"], london_model_data["lat"])
london_model_data = gpd.GeoDataFrame(london_model_data, crs="epsg:4326")

# Check the first rows
london_model_data.head()

,realSum,guest_satisfaction_overall,person_capacity,dist,room_type,lng,lat,geometry
0,570.098074,98.0,2.0,5.301018,Entire home/apt,-0.16032,51.46531,POINT (-0.16032 51.46531)
1,297.984430,99.0,2.0,2.198946,Private room,-0.09683,51.50343,POINT (-0.09683 51.50343)
2,336.790611,96.0,2.0,2.322958,Private room,-0.10554,51.52407,POINT (-0.10554 51.52407)
3,226.722171,99.0,2.0,5.707825,Private room,-0.16575,51.46292,POINT (-0.16575 51.46292)
4,256.355982,98.0,3.0,3.257945,Private room,-0.12055,51.53728,POINT (-0.12055 51.53728)


## Test for Heteroskedasticity Using Pysal OLS

In [15]:
london_OLS = spreg.OLS(london_model_data[['realSum']].values, london_model_data[final_num_predictors].values, 
                  name_y = 'realSum', name_x = final_num_predictors)

print(london_OLS.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:        4367
Mean dependent var  :    296.7700                Number of Variables   :           4
S.D. dependent var  :    170.0585                Degrees of Freedom    :        4363
R-squared           :      0.4387
Adjusted R-squared  :      0.4383
Sum squared residual:70869435.006                F-statistic           :   1136.7726
Sigma-square        :   16243.281                Prob(F-statistic)     :           0
S.E. of regression  :     127.449                Log likelihood        :  -27364.485
Sigma-square ML     :   16228.403                Akaike info criterion :   54736.970
S.E of regression ML:    127.3907                Schwarz criterion     :   54762.498

-----------------------------------------------------------------------------

# Summary

As shown above, the tests for heteroskedasticity will result in a 0.0000 probablilty of accepting the null hypothesis (data is homoskedastic) This means that the data has a form of clustering. It is likely there is more preference for listings nearer to tourist attactions. These listings could have a cluster with their pricings being generally higher than listing with similar features but not near the same tourist attraction. In spatial econometrics, this would be a form of spatial dependence.

This would explain trying to create a linear regression model results in rather inefficient predictions as shown by the low R^2 values for the model, even when trying to maximise the value for both train and test set data while avoid overfitting. 

Therefore, a suggestion for future investigation would be to explore making a model that takes into account and corrects for these spatial factors. In this case, a spatial lag regression model to take into account the spatial dependence of the listings.